In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
from nlpsig_networks.scripts.swmhau_network_functions import (
    swmhau_network_hyperparameter_search
)

In [3]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [4]:
%run ../load_anno_mi.py

In [5]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-08-30 00:00:13
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-30 00:00:24
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-30 00:00:25
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-08-30 00:00:34
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-08-30 00:00:34


In [6]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)
    
sbert_embeddings.shape

(9699, 384)

# swmhau Network

## Obtaining path by looking at post history

We can obtain a path by looking at the history of each post. Here we look at the last 10 posts (and pad with vectors of zeros if there are less than 10 posts) including the current post.

We only want to consider paths that correspond to a client's utterance as we want to model a change in mood at that time. Their history will still contain the therapist's utterances too.

In [7]:
features = ["timeline_index"]
standardise_method = [None]
include_features_in_path = True

In [8]:
num_epochs = 100
dimensions = [15] # [50, 15]
# define swmhau parameters: (output_channels, sig_depth, num_heads)
swmhau_parameters = [(12, 3, 10), (8, 4, 6), (8, 4, 12)]
num_layers = [1]
ffn_hidden_dim_sizes = [[128,128],[256,256],[512,512]]
dropout_rates = [0.5, 0.1]
learning_rates = [1e-3, 1e-4, 5e-4]
seeds = [1, 12, 123]
loss = "focal"
gamma = 2
validation_metric = "f1"
patience = 5

## UMAP

In [9]:
size = 20
swmhau_network_umap_kfold_20, best_swmhau_network_umap_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["umap"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_umap_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: umap
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.
saving results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold.csv
saving the best model results dataframe to CSV for this hyperparameter search in client_talk_type_output/swmhau_network_umap_focal_2_20_kfold_best_model.csv


In [10]:
swmhau_network_umap_kfold_20.groupby(["dimensions",
                                     "output_channels",
                                     "sig_depth",
                                     "num_heads",
                                     "num_layers",
                                     "ffn_hidden_dim",
                                     "learning_rate"]).mean()

/tmp/ipykernel_3280226/767646728.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  swmhau_network_umap_kfold_20.groupby(["dimensions",


accuracy  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.626842   
                                                                         0.0005         0.666840   
                                                                         0.0010         0.666909   
                                                          (512, 512)     0.0001         0.610650   
                                                                         0.0005         0.673829   
                                                                         0.0010         0.669296   
                                     12        1          (256, 256)     0.0001         0.623209   
                                                                         0.0005         0.665663   
                                                                         0.0010         0.671649   
                                                          (512, 512)     0.0001         0.609819   
                                                                         0.0005         0.667808   
                                                                         0.0010         0.672791   
           12              3         10        1          (256, 256)     0.0001         0.630129   
                                                                         0.0005         0.653380   
                                                                         0.0010         0.675905   
                                                          (512, 512)     0.0001         0.621756   
                                                                         0.0005         0.656045   
                                                                         0.0010         0.664383   

                                                                                              f1  \
dimensions output_channels sig_depth num_heads num_layers ffn_hidden_dim learning_rate             
15         8               4         6         1          (256, 256)     0.0001         0.416762   
                                                                         0.0005         0.547235   
                                                                         0.0010         0.536207   
                                                          (512, 512)     0.0001         0.414155   
                                                                         0.0005         0.546260   
                                                                         0.0010         0.537901   
                                     12        1          (256, 256)     0.0001         0.411720   
                                                                         0.0005         0.546419   
                                                                         0.0010         0.544120   
                                                          (512, 512)     0.0001         0.428064   
                                                                         0.0005         0.547320   
                                                                         0.0010         0.537426   
           12              3         10        1          (256, 256)     0.0001         0.449359   
                                                                         0.0005         0.527399   
                                                                         0.0010         0.544527   
                                                          (512, 512)     0.0001         0.456652   
                                                                         0.0005         0.515646   
                                                                         0.0010         0.539384   

                                                                                      

In [11]:
best_swmhau_network_umap_kfold_20

,loss,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_loss,valid_accuracy,...,learning_rate,seed,loss_function,gamma,k_fold,n_splits,augmentation_type,hidden_dim_aug,comb_method,batch_size
0,None,0.668673,0.543704,"[0.7868748008920038, 0.49107531562908147, 0.35...",0.546952,"[0.777218376337319, 0.5040214477211796, 0.3596...",0.540830,"[0.7967741935483871, 0.47877758913412566, 0.34...",None,0.663522,...,0.0005,1,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.682998,0.563479,"[0.7959705788295491, 0.518041237113402, 0.3764...",0.566490,"[0.7891566265060241, 0.5243478260869565, 0.385...",0.560712,"[0.8029032258064516, 0.5118845500848896, 0.367...",None,0.670283,...,0.0005,12,focal,2,True,5,Conv1d,None,concatenation,64
0,None,0.662861,0.546156,"[0.7787087235091179, 0.5161550888529887, 0.343...",0.543866,"[0.7934382323401407, 0.49229583975346686, 0.34...",0.549445,"[0.7645161290322581, 0.5424448217317487, 0.341...",None,0.666981,...,0.0005,123,focal,2,True,5,Conv1d,None,concatenation,64


In [12]:
best_swmhau_network_umap_kfold_20["f1"].mean()

0.5511132300345984

In [13]:
best_swmhau_network_umap_kfold_20["precision"].mean()

0.5524359230328121

In [14]:
best_swmhau_network_umap_kfold_20["recall"].mean()

0.5503287929361177

In [15]:
np.stack(best_swmhau_network_umap_kfold_20["f1_scores"]).mean(axis=0)

array([0.7871847 , 0.50842388, 0.35773111])

In [16]:
np.stack(best_swmhau_network_umap_kfold_20["precision_scores"]).mean(axis=0)

array([0.78660441, 0.50688837, 0.36381499])

In [17]:
np.stack(best_swmhau_network_umap_kfold_20["recall_scores"]).mean(axis=0)

array([0.78806452, 0.51103565, 0.35188621])

## Random Projections

In [ ]:
size = 20
swmhau_network_grp_kfold_20, best_swmhau_network_grp_kfold_20, _, __ = swmhau_network_hyperparameter_search(
    num_epochs=num_epochs,
    df=anno_mi,
    id_column="transcript_id",
    label_column="client_talk_type",
    embeddings=sbert_embeddings,
    y_data=y_data_client,
    output_dim=output_dim_client,
    history_lengths=[size],
    dim_reduce_methods=["gaussian_random_projection"],
    dimensions=dimensions,
    log_signature=True,
    swmhau_parameters=swmhau_parameters,
    num_layers=num_layers,
    ffn_hidden_dim_sizes=ffn_hidden_dim_sizes,
    dropout_rates=dropout_rates,
    learning_rates=learning_rates,
    seeds=seeds,
    loss=loss,
    gamma=gamma,
    features=features,
    standardise_method=standardise_method,
    include_features_in_path=include_features_in_path,
    path_indices=client_index,
    k_fold=True,
    patience=patience,
    validation_metric=validation_metric,
    results_output=f"{output_dir}/swmhau_network_grp_focal_{gamma}_{size}_kfold.csv",
    verbose=False
)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]


##################################################
dimension: 15 | method: gaussian_random_projection
[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

[INFO] Concatenating the embeddings to the dataframe...
[INFO] - columns beginning with 'e' denote the full embddings.
[INFO] - columns beginning with 'd' denote the dimension reduced embeddings.
[INFO] Adding time feature columns into dataframe in `.df`.
[INFO] Adding 'time_encoding' feature...
[INFO] Adding 'time_diff' feature...
[INFO] Adding 'timeline_index' feature...
[INFO] Padding ids and storing in `.df_padded` and `.array_padded` attributes.


  0%|          | 0/9699 [00:00<?, ?it/s]

[INFO] The path was created for each item in the dataframe, by looking at its history, so to include embeddings in the FFN input, we concatenate the embeddings for each sentence / text.


In [ ]:
swmhau_network_grp_kfold_20.groupby(["dimensions",
                                     "output_channels",
                                     "sig_depth",
                                     "num_heads",
                                     "num_layers",
                                     "ffn_hidden_dim",
                                     "dropout_rate",
                                     "learning_rate"]).mean()

In [ ]:
best_swmhau_network_grp_kfold_20

In [ ]:
best_swmhau_network_grp_kfold_20["f1"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["precision"].mean()

In [ ]:
best_swmhau_network_grp_kfold_20["recall"].mean()

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["f1_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["precision_scores"]).mean(axis=0)

In [ ]:
np.stack(best_swmhau_network_grp_kfold_20["recall_scores"]).mean(axis=0)